In [1]:
## packages

# data processing
import numpy as np
import pandas as pd

# modelling
#from sklearn import linear_model
#from sklearn import ensemble
#from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score # metrics
#from sklearn.pipeline import Pipeline
#from sklearn.preprocessing import PolynomialFeatures
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV # hyperparameter tuning

# visualization
from matplotlib import pyplot as plt
import seaborn as sns

In [5]:
final_geo_df = pd.read_csv('2.csv')

final_geo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5363 entries, 0 to 5362
Data columns (total 31 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   date                 5363 non-null   int64  
 1   home_team            5363 non-null   object 
 2   away_team            5363 non-null   object 
 3   country              5363 non-null   object 
 4   country_full_home    5363 non-null   object 
 5   country_full_away    5363 non-null   object 
 6   rank_diff            5363 non-null   float64
 7   point_diff           5363 non-null   float64
 8   point_increase_home  5363 non-null   float64
 9   point_increase_away  5363 non-null   float64
 10  score_diff           5363 non-null   float64
 11  home_won             5363 non-null   int64  
 12  lati_cn_home         5363 non-null   float64
 13  long_cn_home         5363 non-null   float64
 14  lati_cn_away         5363 non-null   float64
 15  long_cn_away         5363 non-null   f

In [58]:
# split out matches in 2022 as test data
import datetime as dt
# data.sort
# final_geo_df['date'] = pd.to_datetime(final_geo_df['date'])
# test = final_geo_df[final_geo_df['date'].dt.year.between(2020, 2022)]
# rest = final_geo_df[final_geo_df['date'].dt.year.between(1993, 2019)]
final_geo_df.sort_values(by=['date'])
rest = final_geo_df.iloc[:4715]
test = final_geo_df.iloc[4715:]

assert len(final_geo_df)==len(test)+len(rest) # if true, split correct
print('test size:', test.shape,'train size', rest.shape)

test size: (648, 31) train size (4715, 31)


In [8]:
# print('Neutral Ratio:', 'Total dataset', round(final_geo_df.neutral.value_counts()/len(final_geo_df),3), 
#       'Train set', round(rest.neutral.value_counts()/len(rest),3),
#       'Test set', round(test.neutral.value_counts()/len(test),3), sep='\n')

AttributeError: 'DataFrame' object has no attribute 'neutral'

In [9]:
test.home_won.value_counts()/len(test) # distribution similar to the overall

 1    0.462963
-1    0.314815
 0    0.222222
Name: home_won, dtype: float64

In [13]:
# Investigate relationships between attributes according to Pearson Correlation Table
# corr_table = round(final_geo_df[['rank_diff', 'point_diff', 'point_increase_home', 'point_increase_away',
#                                  'latitude_home', 'longitude_home','latitude_away', 'longitude_away', 'lati_cn_home', 
#                                  'long_cn_home', 'lati_cn_away', 'long_cn_away']].corr(),2) # Pearson correlation table
# mask = np.triu(np.ones_like(corr_table))
# # Visualize Pearson correlation table in heatmap
# plt.figure(figsize=(12, 8))
# sns.heatmap(corr_table, annot=True, cmap="YlGnBu", mask=mask)

KeyError: "['longitude_home', 'latitude_home', 'latitude_away', 'longitude_away'] not in index"

In [14]:
def roc_auc_score_multiclass(actual_class, pred_class, average = "macro"):
    
    #creating a set of all the unique classes using the actual class list
    unique_class = set(actual_class)
    roc_auc_dict = {}
    for per_class in unique_class:
        
        #creating a list of all the classes except the current class 
        other_class = [x for x in unique_class if x != per_class]

        #marking the current class as 1 and all other classes as 0
        new_actual_class = [0 if x in other_class else 1 for x in actual_class]
        new_pred_class = [0 if x in other_class else 1 for x in pred_class]

        #using the sklearn metrics method to calculate the roc_auc_score
        roc_auc = roc_auc_score(new_actual_class, new_pred_class, average = average)
        roc_auc_dict[per_class] = roc_auc

    return roc_auc_dict

In [15]:
from sklearn.ensemble import StackingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
#from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold

In [17]:
rest['home_won']

0       0
1       0
2      -1
3      -1
4       0
       ..
4710    1
4711    0
4712   -1
4713    1
4714   -1
Name: home_won, Length: 4715, dtype: int64

In [20]:
# train multiclass model
feature = ['rank_diff', 'point_increase_home', 'point_increase_away',
              'latitude_home', 'longitude_home','latitude_away', 'longitude_away', 'lati_cn_home', 
              'long_cn_home', 'lati_cn_away', 'long_cn_away', 
              'neutral_0','neutral_1',
              'rest_days_2.0',
            'rest_days_3.0',
            'rest_days_4.0',
            'rest_days_5.0',
            'rest_days_6.0',
            'rest_days_7.0',
            'rest_days_8.0',
            'rest_days_9.0',
            'rest_days_10.0',
            'rest_days_11.0',
            'rest_days_12.0',
            'rest_days_13.0',
            'rest_days_14.0'
] #dropped point_diff, neutral

X_train, y_train = rest.drop(columns=['home_won']), rest['home_won']

# Feature Scaling
#features = PolynomialFeatures(degree=2)
scaling = StandardScaler()

# trainx_scaled = scaling.fit_transform(X_train)

In [24]:
X_train1 = rest['home_team']

In [25]:
# scaling.fit(X=X_train1,y=y_train)

ValueError: could not convert string to float: 'Angola'

In [26]:
estimators = [('SVM',SVC(decision_function_shape='ovr')),('knn',KNeighborsClassifier()),('xgb',XGBClassifier(objective='multi:softprob'))]
clf = StackingClassifier( estimators=estimators, final_estimator= LogisticRegression(multi_class='multinomial', max_iter=150))
kf = KFold(n_splits=5)

In [27]:
params = {'xgb__max_depth': np.arange(4, 12, 2),
              'xgb__min_child_weight':[5, 10, 15],
              'xgb__eta': [0.1, 0.01],
              'SVM__C':[0.1,1],
              'SVM__kernel':['poly', 'rbf'],
              'knn__n_neighbors':[3,6,9] }

In [56]:
trainx_scaled = X_train.drop(columns = ['home_team','away_team','country','country_full_home','country_full_away','score_diff'])

KeyError: "['home_team' 'away_team' 'country' 'country_full_home' 'country_full_away'] not found in axis"

In [29]:
search = RandomizedSearchCV(estimator=clf, param_distributions=params, random_state=111, n_jobs=-1, cv=kf)
search.fit(trainx_scaled, y_train)

RandomizedSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
                   estimator=StackingClassifier(estimators=[('SVM', SVC()),
                                                            ('knn',
                                                             KNeighborsClassifier()),
                                                            ('xgb',
                                                             XGBClassifier(base_score=None,
                                                                           booster=None,
                                                                           callbacks=None,
                                                                           colsample_bylevel=None,
                                                                           colsample_bynode=None,
                                                                           colsample_bytree=None,
                                                            

In [30]:
print('best score:', search.best_score_,
      'best params:', search.best_params_, sep='\n')

best score:
1.0
best params:
{'xgb__min_child_weight': 5, 'xgb__max_depth': 10, 'xgb__eta': 0.1, 'knn__n_neighbors': 9, 'SVM__kernel': 'poly', 'SVM__C': 1}


In [33]:
X_test, y_test = test.drop(columns=['home_team','away_team','country','country_full_home','country_full_away','home_won']), test['home_won']
testx_scaled = scaling.transform(X_test)

print( 'Training score:', search.score(trainx_scaled, y_train),
      'Validation score:',search.score(testx_scaled, y_test),
      'auc_score:', roc_auc_score_multiclass(y_test, search.predict(testx_scaled)),
      sep='\n')

NotFittedError: This StandardScaler instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [42]:
def get_label(x):
    return x+1

In [43]:
y_train = rest['home_won'].apply(get_label)
y_train

0       1
1       1
2       0
3       0
4       1
       ..
4710    2
4711    1
4712    0
4713    2
4714    0
Name: home_won, Length: 4715, dtype: int64

In [59]:
X_train = rest.drop(columns=['home_team','away_team','country','country_full_home','country_full_away','home_won'])
X_train

,date,rank_diff,point_diff,point_increase_home,point_increase_away,score_diff,lati_cn_home,long_cn_home,lati_cn_away,long_cn_away,...,rest_days_5.0,rest_days_6.0,rest_days_7.0,rest_days_8.0,rest_days_9.0,rest_days_10.0,rest_days_11.0,rest_days_12.0,rest_days_13.0,rest_days_14.0
0,726595200,0.812479,-0.066732,0.295917,0.870000,0.0,0.000000,0.000000,7.812746,11.280970,...,0,0,0,0,0,0,0,0,0,1
1,727113600,1.860582,-0.159256,0.109771,1.700769,0.0,0.000000,0.000000,39.641481,14.262229,...,0,0,0,0,0,0,0,0,0,1
2,727113600,0.812479,-0.086558,0.482063,1.267324,-2.0,0.000000,0.000000,6.764869,7.039490,...,0,0,0,0,0,0,0,0,0,1
3,727200000,1.494578,-0.112994,0.072542,1.158963,-5.0,0.000000,0.000000,24.579227,7.221665,...,0,0,0,0,0,0,0,0,0,1
4,727200000,1.178483,-0.106385,0.295917,1.303445,0.0,0.000000,0.000000,31.406942,28.259260,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4710,1574092800,-0.435261,0.276928,-0.336980,-0.394213,2.0,0.000000,0.000000,9.830749,9.118534,...,1,0,0,0,0,0,0,0,0,0
4711,1574092800,-0.435261,0.343017,0.407605,0.400436,0.0,2.638027,7.440877,4.654750,14.399358,...,0,0,0,0,0,0,0,0,0,1
4712,1574092800,0.196927,-0.162560,-0.485897,0.075352,-1.0,0.880695,6.309593,1.797498,42.951835,...,0,0,0,0,0,0,0,0,0,1
4713,1574092800,-0.684809,0.508238,-0.374209,-0.394213,2.0,0.000000,0.000000,10.241526,71.573051,...,1,0,0,0,0,0,0,0,0,0


In [60]:
# prepare parameters
params_xgb = {'booster': ['gbtree','dart'],
                'max_depth': np.arange(4, 12, 2),
                'min_child_weight': [5, 10, 15],
                'eta': [0.1, 0.01]} 
# build model
tuned_xgb = RandomizedSearchCV(estimator=XGBClassifier(objective='multi:softprob'), param_distributions=params_xgb,
                              n_jobs=-1, cv=5, random_state=111)
# fit model
tuned_xgb.fit(X_train, y_train)

In [48]:
X_test = test.drop(columns=['home_team','away_team','country','country_full_home','country_full_away','home_won','score_diff'])

In [49]:
y_test = test['home_won'].apply(get_label)
y_test

4715    1
4716    2
4717    2
4718    2
4719    2
       ..
5358    0
5359    0
5360    2
5361    1
5362    2
Name: home_won, Length: 648, dtype: int64

In [50]:
model = tuned_xgb.best_estimator_

In [51]:
print('Model:', model,
      'Training score:', model.score(X_train, y_train),
      'Validation score:',model.score(X_test, y_test),
      'auc_score:', roc_auc_score_multiclass(y_test, model.predict(X_test)),
      sep='\n')


Model:
XGBClassifier(base_score=0.5, booster='dart', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False, eta=0.1,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.100000001, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=5,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', random_state=0, ...)
Training score:
1.0
Validation score:
1.0
auc_score:
{0: 1.0, 1: 1.0, 2: 1.0}


In [54]:
model.predict(X_test.iloc[50:55])


array([2, 2, 2, 2, 2])

In [55]:
y_test.iloc[50:55]

4765    2
4766    2
4767    2
4768    2
4769    2
Name: home_won, dtype: int64